In [1]:
%matplotlib ipympl
import numpy as np
from mask_utils import *
import matplotlib.pyplot as plt
from matplotlib import cm

# CONVERT FITS MASK IN DXF

## Reading mask file returning mask image (array) and header

In [2]:
fitsmask = 'mask_050_1040x17_20250709.fits'
#fitsmask = 'wfm_mask.fits'
#fitsmask = 'wfm_mask_test.fits'


In [3]:
with pyfits.open(fitsmask) as hdu_list:     
    header = hdu_list['MASK'].header
    NELE   = header['NAXIS2']
    ELXDIM = header['ELXDIM']
    ELYDIM = header['ELYDIM']
    ELXN   = header['ELXN']
    ELYN   = header['ELYN']
    MINX   = header['MINX']
    MINY   = header['MINY']
    data=Table(hdu_list['MASK'].data)

In [4]:
print(ELXN, ELYN)

1040 522


For each closed element in the mask image, we generate a dxf rectangle with dimension EXLDIM $\times$ ELYDIM

In [5]:
import ezdxf
from ezdxf.addons import r12writer

In [6]:
"""
# Create a new DXF document
doc = ezdxf.new(dxfversion='R2010')
msp = doc.modelspace()


for ele in range(data['X'].size):
    #If it is an open element...
    if data['VAL'][ele] == 1:
        #Define rectangle corners
        x1, y1 = data['X'][ele] - ELXDIM/2, data['Y'][ele] - ELYDIM/2
        x2, y2 = data['X'][ele] + ELXDIM/2, data['Y'][ele] + ELYDIM/2
        msp.add_lwpolyline([(x1, y1),(x2, y1),(x2, y2),(x1, y2),(x1, y1)],close=True)

doc.saveas("wfm_mask_test.dxf")
"""

'\n# Create a new DXF document\ndoc = ezdxf.new(dxfversion=\'R2010\')\nmsp = doc.modelspace()\n\n\nfor ele in range(data[\'X\'].size):\n    #If it is an open element...\n    if data[\'VAL\'][ele] == 1:\n        #Define rectangle corners\n        x1, y1 = data[\'X\'][ele] - ELXDIM/2, data[\'Y\'][ele] - ELYDIM/2\n        x2, y2 = data[\'X\'][ele] + ELXDIM/2, data[\'Y\'][ele] + ELYDIM/2\n        msp.add_lwpolyline([(x1, y1),(x2, y1),(x2, y2),(x1, y2),(x1, y1)],close=True)\n\ndoc.saveas("wfm_mask_test.dxf")\n'

In [7]:
# Create a new DXF document
doc = ezdxf.new(dxfversion='R2010')
msp = doc.modelspace()

#Sorting data by X values

sort_idx = np.lexsort( (data['Y'], data['X']))
data = data[:][sort_idx]


x1, y1 = data['X'][0] - ELXDIM/2, data['Y'][0] - ELYDIM/2
x2, y2 = data['X'][0] + ELXDIM/2, data['Y'][0] + ELYDIM/2

for ele in range(data['X'].size):
    #If it is an open element...
    if data['VAL'][ele] == 1:

        #Check if pixel is adjancent with the previous one
        if (abs(data['X'][ele] - x2) < ELXDIM) and (abs(data['Y'][ele] - y2) < ELYDIM):
            #update only x2, y2 coordinates
            x2, y2 = data['X'][ele] + ELXDIM/2, data['Y'][ele] + ELYDIM/2
        else:
            #Write previous slit and update both corrdinates
            msp.add_lwpolyline([(x1, y1),(x2, y1),(x2, y2),(x1, y2),(x1, y1)],close=True)
            x1, y1 = data['X'][ele] - ELXDIM/2, data['Y'][ele] - ELYDIM/2
            x2, y2 = data['X'][ele] + ELXDIM/2, data['Y'][ele] + ELYDIM/2
#Write last slit
msp.add_lwpolyline([(x1, y1),(x2, y1),(x2, y2),(x1, y2),(x1, y1)],close=True)

doc.saveas("mask_050_1040x17_20250709.dxf")